# Setup

In [1]:
import os, json,re
from tqdm import tqdm 

## Bible books 

In [16]:
corrected_books = {
    'Gen': 'Genesis',
    'Exo': 'Exodus',
    'Lev': 'Leviticus',
    'Num': 'Numbers',
    'Deu': 'Deuteronomy',
    'Jos': 'Joshua',
    'Jdg': 'Judges',
    'Rut': 'Ruth',
    '1Sa': '1 Samuel',
    '2Sa': '2 Samuel',
    '1Ki': '1 Kings',
    '2Ki': '2 Kings',
    '1Ch': '1 Chronicles',
    '2Ch': '2 Chronicles',
    'Ezr': 'Ezra',
    'Neh':'Nehemiah',
    'Est': 'Esther',
    'Job': 'Job',
    'Psa': 'Psalms',
    'Pro': 'Proverbs',
    'Ecc': 'Ecclesiastes',
    'Sng': 'Canticles',
    'Isa': 'Isaiah',
    'Jer': 'Jeremiah',
    'Lam': 'Lamentations',
    'Ezk': 'Ezekiel',
    'Dan': 'Daniel',
    'Hos': 'Hosea',
    'Jol': 'Joel',
    'Amo': 'Amos',
    'Oba': 'Obadiah',
    'Jon': 'Jonah',
    'Mic': 'Micah',
    'Nam': 'Nahum',
    'Hab': 'Habakkuk',
    'Zep': 'Zephaniah',
    'Hag': 'Haggai',
    'Zec': 'Zechariah',
    'Mal': 'Malachi',
    'Mat': 'Matthew',
    'Mrk': 'Mark',
    'Luk': 'Luke',
    'Jhn': 'John',
    'Act': 'Acts',
    'Rom': 'Romans',
    '1Co': '1 Corinthians',
    '2Co': '2 Corinthians',
    'Gal': 'Galatians',
    'Eph': 'Ephesians',
    'Php': 'Philippians',
    'Col': 'Colossians',
    '1Th': '1 Thessalonians',
    '2Th': '2 Thessalonians',
    '1Ti': '1 Timothy',
    '2Ti': '2 Timothy',
    'Tit': 'Titus',
    'Phm': 'Philemon',
    'Heb': 'Hebrews',
    'Jas': 'James',
    '1Pe': '1 Peter',
    '2Pe': '2 Peter',
    '1Jn': '1 John',
    '2Jn': '2 John',
    '3Jn': '3 John',
    'Jud': 'Jude',
    'Rev': 'Revelation',
    'Sus': "Susanna",
    'Bel':"Bel and the Dragon",
    'Bar':'Baruch',
    'Sir':'Ecclesiasticus',
    'Wis':'Wisdom',
    '1Ma':'1 Maccabees',
    '2Ma':'2 Maccabees',
    '1Es':"1 Esdras",
    '2Es':'2 Esdras',
    'Tob':'Tobit',
    'Jdt':'Judith',
    '4Ma':'4 Maccabees',
}

## Load Bibles 

In [95]:
with open("../assets/Bibles/ESV.json") as file: 
    ESV = json.load(file)
    
bible = {}

b_versions = ['AKJV','Geneva', 'ODRV','Douay-Rheims', 'Tyndale', 'Wycliffe','Vulgate']
for bname in b_versions:
    data = pd.read_csv(f"../assets/Bibles/{bname}.csv",header=None)
    data = data.to_dict(orient="records")
    for entry in tqdm(data):
        key = entry[0]
        v_id = key.split(" (")[0]
        text = entry[6]
        if re.search("Douay-Rheims",key):
            if re.sub("Douay-Rheims","ODRV",key) in bible: continue
        if len(text.split(" ")) < 200:
            bible[key] = f"{v_id} {text}"

        parts = re.split(r'(?<=[\.\?]) (?=[A-Z])|(?<=[\!\:\;])', text)
        parts = [re.sub(r'\s+', ' ', p).strip() for p in parts if len(p.strip(" ")) > 0]
        if (len(parts[0].split(" ")) <= 5 or len(parts[-1].split(" ")) <= 5 or re.search(r"\&\w+\;",parts[0])): 
            for pidx, p in enumerate(parts): continue
        elif len(parts) > 1:  
            for pidx, p in enumerate(parts):
              p_id = f"{key} - {pidx}"
              if len(p) == 0: continue
              if re.search(r"\&\w+\;",p) or len(p.split(" ")) <= 5: continue
              bible[p_id] = f"Part {pidx+1} of {v_id}: {p}"
bible_verses = list(bible.values())
bible_ids = list(bible.keys())
id_to_idx = {v_id:idx for idx, v_id in enumerate(bible_ids)}
len(bible_verses)

100%|██████████| 35809/35809 [00:01<00:00, 30751.83it/s]


276627

# Versification

## Get verse mappings

In [17]:
# Biblical entities 
with open(f"../assets/Bibles/TVTMS - Translators Versification Traditions with Methodology for Standardisation for Eng+Heb+Lat+Grk+Others - STEPBible.org CC BY.txt") as file: 
    data = file.readlines()
in_section = False
KJV = None 
Latin = None 
mappings = {} # AKJV to Latin 
abbrev = {}
for idx, line in enumerate(data): 
    line = line.strip()
    if not line:
        continue
    
    if re.search(r"\$[\d\w]+\.\d+\:",line):
        line = line.split("\t")
        KJV, Latin = None, None 
        for idx, item in enumerate(line): 
            if item == "English KJV": 
                KJV = idx 
            elif item == "Latin": 
                Latin = idx 
         
    elif re.search(r"OneToOne\t[\d\w]+\.\d+",line) or re.search("MergedPrevVerse|SubdividedVerse",line):
        if KJV is None or Latin is None: continue
        line = line.split("\t")
        abbrev[line[KJV].split(".")[0]] = None 
        KJV_id = re.findall(r"([\d\w]+)\.(\d+):(\d+)[-]*(.*?)$",line[KJV])
        if len(KJV_id) == 0: continue 
        KJV_id = list(KJV_id[0])
        Latin_id = re.findall(r"([\d\w]+)\.(\d+):(\d+)[-]*(.*?)$",line[Latin])
        if len(Latin_id) == 0: continue 
        Latin_id = list(Latin_id[0])
        if "." in Latin_id[-1]: Latin_id[-1] = ''
        if "." in KJV_id[-1]: KJV_id[-1] = ''
        
        if Latin_id[0] not in corrected_books: continue 
        if KJV_id[0] not in corrected_books: continue 
        
        KJV_id[0] = corrected_books[KJV_id[0]]
        Latin_id[0] = corrected_books[Latin_id[0]]
        final_KJV_id = f"{KJV_id[0]} {KJV_id[1]}.{KJV_id[2]}"
        final_Latin_id = f"{Latin_id[0]} {Latin_id[1]}.{Latin_id[2]}"

        if Latin_id[-1] == "]":
            mappings[final_KJV_id] = None 
            continue 
        
        if len(KJV_id[-1]) == 0: 
            if final_KJV_id == final_Latin_id: continue 
            mappings[final_KJV_id] = final_Latin_id
        else: 
            KJV_nums = [i for i in range(int(KJV_id[-2]),int(KJV_id[-1])+1)]
            Latin_nums = [i for i in range(int(Latin_id[-2]),int(Latin_id[-1])+1)]
            for idx, i in enumerate(KJV_nums): 
                final_KJV_id = f"{KJV_id[0]} {KJV_id[1]}.{i}"
                final_Latin_id = f"{Latin_id[0]} {Latin_id[1]}.{Latin_nums[idx]}"
                if final_KJV_id == final_Latin_id: continue
                mappings[final_KJV_id] = final_Latin_id


In [20]:
with open("../assets/Bibles/verse_mapping.json",'w+') as file: 
    json.dump(mappings, file)

## Parallel Verses

In [87]:
import pandas as pd 
folder = "../.."

fname = "parallel_verses_UNCONFIDENT - HNDPR"
parallel = pd.read_csv(f"{folder}/QP/{fname}.csv").fillna('')
parallel = parallel.to_dict(orient='records')
for item in parallel: 
    item['text'] = item['ESV'] + " " + item['text']

with open(f"{folder}/QP/predictions/{fname}_DSV3.json",'r') as f: 
    responses = json.load(f) 
with open(f"../assets/Bibles/verse_mapping.json",'r') as f: 
    mapping = json.load(f) 

def fix_name(v_id, v2_id):
  if "1 Kings" in v_id and "1 Kings" in v2_id: v_id = "3 Kings" + v_id.split("1 Kings")[-1]
  elif "2 Kings" in v_id and "2 Kings" in v2_id: v_id = "4 Kings" + v_id.split("2 Kings")[-1]
  elif "1 Samuel" in v_id and "1 Samuel" in v2_id: v_id = "1 Kings" + v_id.split("1 Samuel")[-1]
  elif "2 Samuel" in v_id and "2 Samuel" in v2_id: v_id = "2 Kings" + v_id.split("2 Samuel")[-1]
  elif re.search(r"^\d+ Chronicles",v_id): v_id = re.sub(r"Chronicles","Paralipomenon",v_id)
  return v_id

def fix_name_revert(v_id):
  if "3 Kings" in v_id: v_id = "1 Kings" + v_id.split("3 Kings")[-1]
  elif "4 Kings" in v_id: v_id = "2 Kings" + v_id.split("4 Kings")[-1]
  elif "1 Kings" in v_id: v_id = "1 Samuel" + v_id.split("1 Kings")[-1]
  elif "2 Kings" in v_id: v_id = "2 Samuel" + v_id.split("2 Kings")[-1]
  elif re.search(r"^\d+ Paralipomenon",v_id): v_id = re.sub(r"Paralipomenon","Chronicles",v_id)
  return v_id


In [91]:
mapping['Daniel 4.10']

'Daniel 4.7'

### HNDPR Unconfident 

In [ ]:
seen = {}
num_mapped = 0
new_mapping = {}
for idx, item in enumerate(parallel):
    key = "{}; {} ({})".format(item['ESV'], item['Parallel'], item['version'])
    if key in responses: continue 
    if item['ESV'] not in new_mapping: 
      new_mapping[item['ESV']] = []

    p_id = item['Parallel']
    if item['version'] in ['Douay-Rheims','ODRV','Vulgate','Wycliffe']: 
       p_id = fix_name_revert(p_id)
    
    full_id = item['Parallel'] + f" ({item['version']})"

    if item['ESV'] == p_id and item['version'] in ['AKJV','Geneva','Tyndale']: 
      seen[item['ESV']] = True 
      new_mapping[item['ESV']].append(full_id)
    elif item['ESV'] in mapping:
      if item['version'] in ['Douay-Rheims','ODRV','Vulgate','Wycliffe']: 
        if mapping[item['ESV']] is None:
          seen[item['ESV']] = True
          # there is no equivalent in the Latin-based Bibles  
        elif p_id == mapping[item['ESV']]: 
          new_mapping[item['ESV']].append(full_id)
          seen[item['ESV']] = True  
    elif item['ESV'] == p_id: 
        new_mapping[item['ESV']].append(full_id)
        seen[item['ESV']] = True 

for key in responses: 
  key = key.split("; ")[0]
  seen[key] = True 

not_seen = {}
for idx, item in enumerate(parallel):
    key = "{}; {} ({})".format(item['ESV'], item['Parallel'], item['version'])
    if key in responses: continue 
    if item['ESV'] not in seen: 
      not_seen[(item['ESV'],item['version'])] = True 

for ESV_id, version in not_seen.keys(): 
  if item['ESV'] not in new_mapping: 
    new_mapping[item['ESV']] = []
  if ESV_id not in mapping or version in ['AKJV','Geneva','Tyndale']: 
    new_mapping[ESV_id].append(ESV_id + f" ({item['version']})")
  elif ESV_id in mapping:
    if mapping[ESV_id] is None: continue 
    new_mapping[ESV_id].append(mapping[ESV_id] + f" ({item['version']})")

new_mapping = {k:v for k,v in new_mapping.items() if len(v) >0}
with open(f"../assets/QP_Datasets/parallel_mapping_remainder.json",'w+') as file: 
    json.dump(new_mapping,file)
len(new_mapping)

### All 

In [88]:
def get_parallel_data():
    p_data = {}
    with open(f"{folder}/Early-Modern-Sermons/assets/QP_Datasets/confident_labeled.json",'r') as file:
      labeled_dict = json.load(file)
    for ESV_id, pv_list in labeled_dict.items():
      pv_list = list(set(pv_list))
      p_data[ESV_id] = {'qid': ESV_id,
                        'pos':{k:None for k in pv_list}, # verse ids with version,
                        }
    with open(f"{folder}/Early-Modern-Sermons/assets/QP_Datasets/parallel_mapping_remainder.json",'r') as file: 
      parallel = json.load(file)
    for v_id,vlist in parallel.items():
      if v_id in labeled_dict: continue
      if v_id not in p_data:
        p_data[v_id] = {'qid': v_id,
                        'pos':{}, # high similarity and equivalent numberings
                        } 
      for v in vlist: 
        p_data[v_id]['pos'][v] = None 
    
    training_set = {}
    for key, entry in p_data.items():
      entry['pos'] = list(set(entry['pos']))
      if len(entry['pos']) == 0: continue
      training_set[key] = entry
    return training_set

parallel = get_parallel_data()
len(parallel) # ESV_id to positives

31129

In [89]:
mismatch = {}
from tqdm import tqdm
for ESV_id, v_list in tqdm(parallel.items()): 
    found = {}
    miss = []
    for v in v_list['pos']: 
        v_id = v.split(" (")[0]
        ver = v.split(" (")[-1].strip(")")
        if ver in ['Douay-Rheims', 'Vulgate', 'Wycliffe','ODRV']: 
            v_id_std = fix_name_revert(v_id) 
        else: 
            v_id_std = v_id 

        if ESV_id not in mapping and v_id_std == ESV_id: 
            found[ver] = None 
            continue 
        elif ESV_id not in mapping and v_id_std != ESV_id: 
            miss.append((v_id,ver))
        
        elif ESV_id in mapping and ver in ['AKJV','Geneva','Tyndale']: 
            if v_id_std == ESV_id: 
                found[ver] = None 
            else: 
                miss.append((v_id, ver))
        
        elif ESV_id in mapping:
            if v_id_std == mapping[ESV_id]: 
                found[ver] = None 
                continue 
            elif v_id == mapping[ESV_id]: 
                found[ver] = None 
                continue 
            else: 
                miss.append((v_id, ver))
    for v_id,ver in miss: 
        if ver in found: continue 
        mismatch[(ESV_id, v_id)] = ver
len(mismatch) # 1595 now 1496 (100 in the ones I looked at)

100%|██████████| 31129/31129 [00:00<00:00, 109502.37it/s]


3239

# Proper Nouns 

In [ ]:
# Biblical entities 
with open(f"../assets/Bibles/TIPNR - Translators Individualised Proper Names with all References - STEPBible.org CC BY.txt") as file: 
    data = file.readlines()
in_entities_section = False
e_to_v = {}
for idx, line in enumerate(data): 
    line = line.strip()
    if not line:
        continue
    
    if line.startswith('$========== '):
        in_entities_section = True
        continue  
    
    if idx < 112: continue
    if idx > 14394: continue
    if in_entities_section:
        if line[0] != "–": continue 
        if not re.search("@",line): continue 
        name = re.findall("^(.*?)@",line)[0]
        refs = line.split("reference=")[-1].split("\t")[-1]
        name = name.split("\t")[-1].split("|")
        KJV_name = re.findall(r"KJV\s*=\s*(.*?)[);,]",line)
        if len(KJV_name) > 0: 
            KJV_name = KJV_name[0]
            name.append(KJV_name)
        for n in name: 
            n = re.sub("_"," ",n)
            if n not in e_to_v: e_to_v[n] = []
            ref_list = [re.findall("([\d\w]+)\.(\d+\.\d+)",r) for r in refs.split("; ")]
            ref_list = [f"{corrected_books[r[0][0]]} {r[0][1]}" for r in ref_list if len(r) > 0 if r[0][0] != 'Etc']
            e_to_v[n].extend(sorted(list(set(ref_list))))
    else:
        continue

In [ ]:
with open(f"../assets/Bibles/proper_nouns.json",'w+') as file: 
    json.dump(e_to_v, file)

# Citations 

In [6]:
import pandas as pd
from tqdm import tqdm  
def clean_text(s): 
    s = re.sub(r"\<\/i\>|\<NOTE\>|NONLATINALPHABET|\<i\>|\d+\^PAGE[S]*\^MISSING","",s)
    s = re.sub(r"\s+"," ",s)
    s = s.strip(" ")
    return s 
all_c = {}
c_repo = "../../CITATIONS"
c_files = sorted(k for k in os.listdir(c_repo) if ".csv" in k)
for file in tqdm(c_files): 
    era, prefix = file.split("_citations.csv")[0].split("_")
    # print(era,prefix)
    data = pd.read_csv(f"{c_repo}/{file}",header=None).to_dict(orient='records')
    data = {(d[0],d[1],d[2]):d for d in data}
    text = pd.read_csv(f"../../SERMONS_APP/db/data/{era}/{prefix}_body.csv",header=None).to_dict(orient='records')
    text = {(d[0],d[1],"In-Text"):clean_text(d[6]) for d in text}
    for key, t in text.items(): 
        if key in data: 
            data[key]['text'] = t 
    if f"{prefix}_margin.csv" in os.listdir(f"../../SERMONS_APP/db/data/{era}"): 
        text = pd.read_csv(f"../../SERMONS_APP/db/data/{era}/{prefix}_margin.csv",header=None).to_dict(orient='records')
        text = {(d[0],d[1],"Note " + str(d[2])):clean_text(d[3]) for d in text}
        for key, t in text.items(): 
            if key in data: 
                data[key]['text'] = t
    for d in data.values(): 
        all_c[d[6]] = d[4] 
    # data = pd.DataFrame(list(data.values()))
    # data.to_csv(f"{c_repo}/{file}",index=False)

  4%|▍         | 3/76 [00:16<07:02,  5.79s/it]/var/folders/3_/ylrg8wdj20l755p4921q0wg80000gp/T/ipykernel_11556/2553457035.py:16: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  text = pd.read_csv(f"../../SERMONS_APP/db/data/{era}/{prefix}_body.csv",header=None).to_dict(orient='records')
 36%|███▌      | 27/76 [01:40<01:25,  1.75s/it]/var/folders/3_/ylrg8wdj20l755p4921q0wg80000gp/T/ipykernel_11556/2553457035.py:16: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  text = pd.read_csv(f"../../SERMONS_APP/db/data/{era}/{prefix}_body.csv",header=None).to_dict(orient='records')
 84%|████████▍ | 64/76 [03:23<00:39,  3.29s/it]/var/folders/3_/ylrg8wdj20l755p4921q0wg80000gp/T/ipykernel_11556/2553457035.py:16: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  text = pd.read_csv(f"../../SERMONS_APP/db/data/{era}/{prefix}_body.csv",header=None

In [7]:
list(all_c.items())[1:10]

[('Hosea 11. 11.', 'Hosea 11.11'),
 ('Zek. 16.', 'Ezekiel 16'),
 ('Cant. 5.', 'Canticles 5'),
 ('Psalms 18.', 'Psalms 18'),
 ('Psalms 91.', 'Psalms 91'),
 ('John 15. 4:', 'John 15.4'),
 ('Psalms 18. 2.', 'Psalms 18.2'),
 ('Heb. 1.', 'Hebrews 1'),
 ('Isaiah 56.', 'Isaiah 56')]

In [8]:
import sys 
sys.path.append('../')
from lib.EEPS_helper import isNumeral
from collections import Counter

In [9]:
formats = {}

def space_punctuation(text):
    text = re.sub(r'([\.\,\:\;\!\?\(\)\-\&])(?=\w)', r'\1 ', text)
    text = re.sub(r'(?<=\w)([\.\,\:\;\!\?\(\)\-\&])', r' \1', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

for orig in tqdm(all_c): 
    if orig == '6': continue 
    text = space_punctuation(orig)
    tokens = text.split(" ")
    format = []
    for t in tokens: 
        if isNumeral(t): 
            format.append('N')
        elif re.search(r"[\.\,\:\;\!\?\(\)\-\&]",t):  
            format.append(t)
        else: 
            format.append('W')
    format = " ".join(format)
    if format not in formats: formats[format] = 0
    formats[format] += 1       

100%|██████████| 306017/306017 [00:13<00:00, 21964.18it/s]


In [10]:
Counter(formats).most_common(n=10)

[('W . N . N .', 77457),
 ('W N . N .', 26984),
 ('W . N . N , N .', 15415),
 ('W . N . N', 13070),
 ('W . N . N . N .', 12395),
 ('N W . N . N .', 11083),
 ('W . N .', 8721),
 ('N . W . N . N .', 6718),
 ('W . N N .', 4893),
 ('W . N . N .)', 4872)]

In [14]:
all_books = {}
for orig, std in all_c.items(): 
    if orig == '6' and std == '4': continue 
    words = re.findall(r"([A-Za-z]+)",orig)
    book = re.findall(r"[A-Za-z]+",std)[0].lower()
    if book not in all_books: 
        all_books[book] = {}
    for w in words: 
        if isNumeral(w): 
            continue 
        w = w.lower()
        if w in ['of','cap','chap','verse','vers','chapter','ch']:
            continue 
        if w not in all_books[book]: 
            all_books[book][w] = 0 
        all_books[book][w] += 1 

common_abbrev = {}
for book, abbrevs in all_books.items(): 
    if book == "children": continue 
    top = Counter(abbrevs).most_common(n=1)[0]
    common_abbrev[book] = top[0] 

with open(f"../assets/Bibles/top_abbrev.json",'w+') as file: 
    json.dump(common_abbrev, file)
common_abbrev 

{'canticles': 'cant',
 'hosea': 'hos',
 'ezekiel': 'ezek',
 'psalms': 'psal',
 'john': 'john',
 'hebrews': 'heb',
 'isaiah': 'isa',
 'romans': 'rom',
 'philippians': 'phil',
 'corinthians': 'cor',
 'matthew': 'mat',
 'jeremiah': 'jer',
 'proverbs': 'prov',
 'acts': 'acts',
 'kings': 'king',
 'verse': 'ver',
 'revelation': 'rev',
 'luke': 'luke',
 'chronicles': 'chron',
 'timothy': 'tim',
 'james': 'jam',
 'peter': 'pet',
 'titus': 'tit',
 'jude': 'iud',
 'ephesians': 'eph',
 'colossians': 'col',
 'galatians': 'gal',
 'deuteronomy': 'deut',
 'samuel': 'sam',
 'job': 'job',
 'genesis': 'gen',
 'micah': 'micah',
 'lamentations': 'lam',
 'leviticus': 'levit',
 'exodus': 'exod',
 'zephaniah': 'zeph',
 'wisdom': 'wisd',
 'daniel': 'dan',
 'judges': 'judg',
 'joshua': 'josh',
 'mark': 'mark',
 'habakkuk': 'hab',
 'thessalonians': 'thes',
 'esther': 'es',
 'ecclesiastes': 'eccles',
 'zechariah': 'zach',
 'ezra': 'ezra',
 'amos': 'amos',
 'malachi': 'mal',
 'numbers': 'numb',
 'haggai': 'hag',
